In [1]:
from sentence_transformers import SentenceTransformer

/home/heil/llama-all-local/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = [
    "this is one example sentence",
    "This is a sentence about pasta"
    ]

# Using sentence-transformers

In [3]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
embeddings = model.encode(sentences)
embeddings.shape

(2, 384)

# Using just Huggingface

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [6]:
torch.cuda.is_available()

True

In [7]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min = 1e-9)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [9]:
encoded_input = tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')

In [10]:
with torch.no_grad():
    model_output = model(**encoded_input)

In [11]:
encoded_input

{'input_ids': tensor([[  101,  2023,  2003,  2028,  2742,  6251,   102,     0],
        [  101,  2023,  2003,  1037,  6251,  2055, 24857,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
x = model_output[0]
y = encoded_input["attention_mask"]

y.unsqueeze(-1).expand(x.size()).shape

torch.Size([2, 8, 384])

In [13]:
y

tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [14]:
torch.sum(x, 1).shape

torch.Size([2, 384])

In [15]:
mean_pooling(model_output, y).shape

torch.Size([2, 384])